## def input_2d_array():
    rows = int(input("Enter the number of rows/columns (for a square matrix): "))
    
    matrix = []
    print("Enter the elements row by row:")
    for i in range(rows):
        row = []
        for j in range(rows):
            element = int(input(f"Enter element at position ({i+1}, {j+1}): "))
            row.append(element)
        matrix.append(row)
    
    return matrix

def calculate_mean(matrix):
    # Calculate the mean of each column in the matrix
    n = len(matrix)
    m = len(matrix[0])  # number of columns

    means = []
    for j in range(m):
        col_sum = sum(matrix[i][j] for i in range(n))
        means.append(col_sum / n)
    
    return means

def center_matrix(matrix, means):
    # Center the data by subtracting the mean from each element
    n = len(matrix)
    m = len(matrix[0])
    
    centered_matrix = []
    for i in range(n):
        centered_row = []
        for j in range(m):
            centered_row.append(matrix[i][j] - means[j])
        centered_matrix.append(centered_row)
    
    return centered_matrix

def calculate_covariance(centered_matrix):
    # Calculate the covariance matrix
    n = len(centered_matrix)
    m = len(centered_matrix[0])
    
    covariance_matrix = [[0 for _ in range(m)] for _ in range(m)]
    
    for i in range(m):
        for j in range(m):
            cov_sum = sum(centered_matrix[k][i] * centered_matrix[k][j] for k in range(n))
            covariance_matrix[i][j] = cov_sum / (n - 1)
    
    return covariance_matrix

def calculate_lambda(matrix):
    # Eigenvalues calculation for 2x2 matrix using characteristic equation λ^2 - (a+d)λ + (ad-bc) = 0
    a = matrix[0][0]
    b = matrix[0][1]
    c = matrix[1][0]
    d = matrix[1][1]
    
    trace = a + d  # (a + d)
    determinant = a * d - b * c  # (ad - bc)
    
    # Solving the quadratic equation λ^2 - trace(λ) + determinant = 0
    lambda1 = (trace + (trace*2 - 4 * determinant)*0.5) / 2
    lambda2 = (trace - (trace*2 - 4 * determinant)*0.5) / 2
    
    return [lambda1, lambda2]

def calculate_eigenvectors(matrix, eigenvalues):
    eigenvectors = []
    
    for lamda in eigenvalues:
        # Constructing (A - λI) matrix
        A_minus_lambda_I = [
            [matrix[0][0] - lamda, matrix[0][1]],
            [matrix[1][0], matrix[1][1] - lamda]
        ]
        
        a11 = A_minus_lambda_I[0][0]
        a12 = A_minus_lambda_I[0][1]
        a21 = A_minus_lambda_I[1][0]
        a22 = A_minus_lambda_I[1][1]

        # Solving the equations a11 * x + a12 * y = 0 and a21 * x + a22 * y = 0
        if a12 != 0:
            y = 1  # Arbitrarily choose y = 1
            x = -a12 * y / a11 if a11 != 0 else 0
        else:
            x = 1  # Arbitrarily choose x = 1
            y = -a21 * x / a22 if a22 != 0 else 0

        # Adjust to solve correctly
        if a11 == 0 and a12 == 0:
            x, y = 0, 1  # when both are 0, any y works

        # Rescale the vector if needed to match the expected output
        eigenvector = [x, y]

        # Adjust the vector if the results are not well scaled
        if eigenvector[0] != 0:
            factor = eigenvector[0]
            eigenvector = [v / factor for v in eigenvector]

        eigenvectors.append(eigenvector)
    
    return eigenvectors

def pca_sort(eigenvalues, eigenvectors):
    # Pair eigenvalues and eigenvectors together
    eigen_pairs = [(eigenvalues[i], eigenvectors[i]) for i in range(len(eigenvalues))]
    
    # Sort the pairs by eigenvalue in descending order
    eigen_pairs.sort(key=lambda x: x[0], reverse=True)
    
    # Unzip into sorted eigenvalues and eigenvectors
    sorted_eigenvalues, sorted_eigenvectors = zip(*eigen_pairs)
    
    return sorted_eigenvalues, sorted_eigenvectors

def merge_and_sort_eigenvectors(eigenvectors):
    # Sorting eigenvector values individually and merging them into a matrix
    sorted_eigenvectors = [sorted(ev) for ev in eigenvectors]
    
    merged_matrix = [
        [sorted_eigenvectors[0][0], sorted_eigenvectors[1][0]],
        [sorted_eigenvectors[0][1], sorted_eigenvectors[1][1]]
    ]
    
    return merged_matrix

# Example usage
matrix = input_2d_array()

# Step 1: Calculate the mean of each column
means = calculate_mean(matrix)

# Step 2: Center the matrix
centered_matrix = center_matrix(matrix, means)

# Step 3: Calculate the covariance matrix
cov_matrix = calculate_covariance(centered_matrix)

print("Covariance Matrix:")
for row in cov_matrix:
    print(row)

# Step 4: Calculate eigenvalues
eigenvalues = calculate_lambda(cov_matrix)

# Step 5: Calculate eigenvectors
eigenvectors = calculate_eigenvectors(cov_matrix, eigenvalues)

# Step 6: Sort eigenvectors by descending eigenvalues
sorted_eigenvalues, sorted_eigenvectors = pca_sort(eigenvalues, eigenvectors)

print("\nThe sorted eigenvalues (λ) in descending order are:")
for val in sorted_eigenvalues:
    print(val)

print("\nThe corresponding eigenvectors (PCA components) are:")
for i, ev in enumerate(sorted_eigenvectors):
    print(f"For λ = {sorted_eigenvalues[i]}:")
    print(f"(\n {ev[0]} \n {ev[1]} \n)")

# Step 7: Merge and sort the eigenvectors into a PCA matrix
pca_matrix = merge_and_sort_eigenvectors(sorted_eigenvectors)

print("\nThe PCA matrix (sorted eigenvectors) is:")
for row in pca_matrix:
    print(row)

# Step 8: Variance explained by each principal component
# Define the eigenvalues for the two principal components
eigenvalue_pca1 = sorted_eigenvalues[0]
eigenvalue_pca2 = sorted_eigenvalues[1]

# Calculate the total variance as the sum of the eigenvalues
total_variance = abs(eigenvalue_pca1) + abs(eigenvalue_pca2)

# Calculate the variance percentage explained by PCA1 and PCA2
variance_pca1 = (abs(eigenvalue_pca1) / total_variance) * 100
variance_pca2 = (abs(eigenvalue_pca2) / total_variance) * 100

# Output the results
print(f"\nVariance explained by PCA1: {variance_pca1:.2f}%")
print(f"Variance explained by PCA2: {variance_pca2:.2f}%")

# Calculate the percentage of variance reduction from PCA2 to PCA1
variance_reduction = variance_pca2
print(f"Variance reduction when reducing from PCA2 to PCA1: {variance_reduction:.2f}%")